Recitation 2
============

In this recitation, we will perform an MD5 length extention attack. We will use library functions from the pymd5.py file from Canvas, so begin by downloading that and saving it in the same folder as the notebook.

Imagine a web service that allows users to pass multiple commands to a service using URL parameters (the values following the `?`, separated by `&`):

https://example.com/api?token=1e755d78dcb4d783b2573b8d04fcc48a&user=admin&command1=ListFiles&command2=NoOp

Here the `user` parameter is the current user, and there can be any number of `command1`, `command2`, etc. parameters to store the various commands sent to the server. The `token` parameter is used to authenticate the command, and is generated using `MD5([user’s 8-character password] + [the rest of the URL starting from user= and ending with the last command])`.

Imagine an attacker wants to add a new command parameter, `command3=DeleteAllData`. They cannot simply append it to the query, as that would invalidate the token. They cannot generate a new token from scratch, as they need the user's password. However, they can use a **length extension attack** to extend the existing token with their new command. Your goal is to demonstrate this attack by using the code below to calculate a valid URL with the parameter `command3=DeleteAllData`.

_NOTE: The code below is what the students will be given_

In [3]:
import urllib

import pymd5

url = 'https://example.com/api?token=1e755d78dcb4d783b2573b8d04fcc48a&user=admin&command1=ListFiles&command2=NoOp'

original_token = '1e755d78dcb4d783b2573b8d04fcc48a'
original_query = 'user=admin&command1=ListFiles&command2=NoOp'
malicious_extension = '&command3=DeleteAllFiles'

# We cannot simply append to the end of the original query, as the original MD5
# hash is based off the assumption that the message ends there, and some padding
# has been added at the end. We will have to calculate that padding, and add it
# before our malicious extension. This padding is based off the original message
# length, so we need to calculate that first.

# To calculate the padding, we need to know the message length. We also need to
# account for the user's 8-character password, which we do not see, as well as
# the length of the query which we do!
original_message_length = len(original_query) + 8
# The pymd5.padding function takes the padding length in *bits*, not *bytes*,
# so we will have to convert when calculating it.
message_padding = pymd5.padding(original_message_length * 8)

# To recreate the state of the original message, we need the total message
# length which includes both the original message, and the padding. This is
# also in *bits*!
total_message_length = (original_message_length + len(message_padding)) * 8

# We now have enough information to recreate the state of the hash function at
# the point where it left off originally.
h = pymd5.md5(state=bytes.fromhex(original_token), count=total_message_length)

# We can now calculate the hash H(m + a), where m is the orignal message (plus
# padding!) and a is any new text that needs to be added. The h.update function
# takes the additional text to append to the end.
h.update(malicious_extension)

updated_token = h.hexdigest()

# Finally, the padding contains null and unprintable characters, so we need to
# escape it so we can insert it as part of a URL.
url_safe_padding = urllib.parse.quote(message_padding)

# We now need to put the token, the original text, the escaped padding, and our
# new addition back together into a final URL, in the same form as the orignal.
updated_query = 'token={}&{}{}{}'.format(updated_token, original_query, url_safe_padding, malicious_extension)

print('https://example.com/api?{}'.format(updated_query))

https://example.com/api?token=d037150d6ddc74fff8db6e4d6d5bbf28&user=admin&command1=ListFiles&command2=NoOp%80%00%00%00%00%98%01%00%00%00%00%00%00&command3=DeleteAllFiles


The correct answer should have a token of `d037150d6ddc74fff8db6e4d6d5bbf2`